# Library

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import norm, skew, kurtosis, jarque_bera
from scipy import stats
from arch import arch_model
from statsmodels.stats.diagnostic import het_arch

# Data

In [2]:
tickers = ['AAPL', 'AMZN', 'META', 'TSLA', 'TSM', 'IBM', 'BABA']

In [3]:
# Mendapatkan data saham TSMC (NYSE: TSM)
stocks_df = yf.download(tickers, start="2023-12-01", end="2024-11-30")['Close']

# Beri nama kolom agar lebih mudah dikenali
# stocks_df.columns = ['AAPL', 'AMZN', 'META', 'TSLA', 'TSM', 'IBM']

stocks_df.index = pd.to_datetime(stocks_df.index)

# Tampilkan DataFrame yang telah digabung
stocks_df

[*********************100%***********************]  7 of 7 completed


Ticker,AAPL,AMZN,BABA,IBM,META,TSLA,TSM
Date,,,,,,,
2023-12-01,191.240005,147.029999,73.989998,160.550003,324.820007,238.830002,98.550003
2023-12-04,189.429993,144.839996,73.029999,161.100006,320.019989,235.580002,97.010002
2023-12-05,193.419998,146.880005,72.379997,161.389999,318.290009,238.720001,96.690002
2023-12-06,192.320007,144.520004,71.489998,160.279999,317.450012,239.369995,97.849998
2023-12-07,194.270004,146.880005,72.330002,160.220001,326.589996,242.639999,99.290001
...,...,...,...,...,...,...,...
2024-11-22,229.869995,197.119995,83.129997,222.970001,559.140015,352.559998,190.080002
2024-11-25,232.869995,201.449997,85.580002,226.130005,565.109985,338.589996,185.080002
2024-11-26,235.059998,207.860001,85.180000,228.830002,573.539978,338.230011,183.839996


# Log Return

In [4]:
# log_returns =  np.log(stocks_df / stocks_df.shift(1)).dropna()
log_returns =  np.log(stocks_df / stocks_df.shift(1)).dropna()
log_returns

Ticker,AAPL,AMZN,BABA,IBM,META,TSLA,TSM
Date,,,,,,,
2023-12-04,-0.009510,-0.015007,-0.013060,0.003420,-0.014888,-0.013701,-0.015750
2023-12-05,0.020844,0.013986,-0.008940,0.001798,-0.005421,0.013241,-0.003304
2023-12-06,-0.005703,-0.016198,-0.012372,-0.006902,-0.002643,0.002719,0.011926
2023-12-07,0.010088,0.016198,0.011681,-0.000374,0.028385,0.013568,0.014609
2023-12-08,0.007385,0.003670,-0.002630,0.010802,0.018686,0.004933,0.010619
...,...,...,...,...,...,...,...
2024-11-22,0.005890,-0.006372,-0.029046,0.002560,-0.007040,0.037335,-0.006084
2024-11-25,0.012966,0.021729,0.029046,0.014073,0.010620,-0.040431,-0.026657
2024-11-26,0.009360,0.031324,-0.004685,0.011869,0.014807,-0.001064,-0.006722


In [5]:
log_returns.index = pd.to_datetime(log_returns.index)

In [6]:
# Gabungkan log returns dengan stocks_df_cut
combined_df = pd.concat([stocks_df, log_returns], axis=1)

# Ubah nama kolom agar lebih jelas
combined_df.columns = [f'{col}_Close' if i < len(stocks_df.columns) else f'{col}_LogReturn' 
                       for i, col in enumerate(stocks_df.columns.append(log_returns.columns))]

# Tampilkan DataFrame yang sudah digabung
combined_df

,AAPL_Close,AMZN_Close,BABA_Close,IBM_Close,META_Close,TSLA_Close,TSM_Close,AAPL_LogReturn,AMZN_LogReturn,BABA_LogReturn,IBM_LogReturn,META_LogReturn,TSLA_LogReturn,TSM_LogReturn
Date,,,,,,,,,,,,,,
2023-12-01,191.240005,147.029999,73.989998,160.550003,324.820007,238.830002,98.550003,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-04,189.429993,144.839996,73.029999,161.100006,320.019989,235.580002,97.010002,-0.009510,-0.015007,-0.013060,0.003420,-0.014888,-0.013701,-0.015750
2023-12-05,193.419998,146.880005,72.379997,161.389999,318.290009,238.720001,96.690002,0.020844,0.013986,-0.008940,0.001798,-0.005421,0.013241,-0.003304
2023-12-06,192.320007,144.520004,71.489998,160.279999,317.450012,239.369995,97.849998,-0.005703,-0.016198,-0.012372,-0.006902,-0.002643,0.002719,0.011926
2023-12-07,194.270004,146.880005,72.330002,160.220001,326.589996,242.639999,99.290001,0.010088,0.016198,0.011681,-0.000374,0.028385,0.013568,0.014609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-22,229.869995,197.119995,83.129997,222.970001,559.140015,352.559998,190.080002,0.005890,-0.006372,-0.029046,0.002560,-0.007040,0.037335,-0.006084
2024-11-25,232.869995,201.449997,85.580002,226.130005,565.109985,338.589996,185.080002,0.012966,0.021729,0.029046,0.014073,0.010620,-0.040431,-0.026657
2024-11-26,235.059998,207.860001,85.180000,228.830002,573.539978,338.230011,183.839996,0.009360,0.031324,-0.004685,0.011869,0.014807,-0.001064,-0.006722


In [7]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2023-12-01 to 2024-11-29
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AAPL_Close      251 non-null    float64
 1   AMZN_Close      251 non-null    float64
 2   BABA_Close      251 non-null    float64
 3   IBM_Close       251 non-null    float64
 4   META_Close      251 non-null    float64
 5   TSLA_Close      251 non-null    float64
 6   TSM_Close       251 non-null    float64
 7   AAPL_LogReturn  250 non-null    float64
 8   AMZN_LogReturn  250 non-null    float64
 9   BABA_LogReturn  250 non-null    float64
 10  IBM_LogReturn   250 non-null    float64
 11  META_LogReturn  250 non-null    float64
 12  TSLA_LogReturn  250 non-null    float64
 13  TSM_LogReturn   250 non-null    float64
dtypes: float64(14)
memory usage: 29.4 KB


In [8]:
# Ekspor DataFrame ke file CSV
combined_df.to_csv('stocks_and_log_returns.csv')

# Uji Normalitas Return

## Kolmogorov-Smirnov

In [9]:
# Langkah 3: Uji Normalitas (Kolmogorov-Smirnov)
def kolmogorov_smirnov_test(data):
    # Uji Kolmogorov-Smirnov terhadap distribusi normal
    stat, p_value = kstest(data, 'norm', args=(data.mean(), data.std()))
    print(f"KS Test Statistic: {stat}, p-value: {p_value}")
    if p_value < 0.05:
        print("Data tidak berdistribusi normal")
    else:
        print("Data berdistribusi normal")
    print("")

# Uji normalitas untuk masing-masing saham
for ticker in tickers:
    print(f"Uji Normalitas untuk {ticker}")
    kolmogorov_smirnov_test(log_returns[ticker])


Uji Normalitas untuk AAPL


NameError: name 'kstest' is not defined

## Shapiro-Wilk

In [ ]:
# Fungsi untuk uji normalitas Shapiro-Wilk
def shapiro_wilk_test(data):
    stat, p_value = stats.shapiro(data)
    print("Shapiro-Wilk Test:")
    print(f"Statistic: {stat}, p-value: {p_value}")
    if p_value > 0.05:
        print("Data mengikuti distribusi normal (p > 0.05).")
    else:
        print("Data tidak mengikuti distribusi normal (p ≤ 0.05).")
    print("")

for ticker in tickers:
    print(f"Uji Normalitas untuk {ticker}")
    shapiro_wilk_test(log_returns[ticker])

Uji Normalitas untuk AAPL
Shapiro-Wilk Test:
Statistic: 0.9581525991793911, p-value: 1.2055024058804733e-06
Data tidak mengikuti distribusi normal (p ≤ 0.05).

Uji Normalitas untuk AMZN
Shapiro-Wilk Test:
Statistic: 0.9593624015429414, p-value: 1.6818545625344084e-06
Data tidak mengikuti distribusi normal (p ≤ 0.05).

Uji Normalitas untuk META
Shapiro-Wilk Test:
Statistic: 0.8374883515654734, p-value: 1.7440736944267994e-15
Data tidak mengikuti distribusi normal (p ≤ 0.05).

Uji Normalitas untuk TSLA
Shapiro-Wilk Test:
Statistic: 0.9407013336320122, p-value: 1.6336812238552717e-08
Data tidak mengikuti distribusi normal (p ≤ 0.05).

Uji Normalitas untuk TSM
Shapiro-Wilk Test:
Statistic: 0.9767738070578906, p-value: 0.0004103377659306081
Data tidak mengikuti distribusi normal (p ≤ 0.05).

Uji Normalitas untuk IBM
Shapiro-Wilk Test:
Statistic: 0.868340637765407, p-value: 7.7819929503987e-14
Data tidak mengikuti distribusi normal (p ≤ 0.05).

Uji Normalitas untuk BABA
Shapiro-Wilk Test:
St

## Jarque-Bera

In [ ]:
# Langkah 3: Uji Normalitas (Jarque-Bera)
def jarque_bera_test(data):
    stat, p_value = jarque_bera(data)
    print(f"JB Test Statistic: {stat}, p-value: {p_value}")
    if p_value < 0.05:
        print("Data tidak berdistribusi normal")
    else:
        print("Data berdistribusi normal")
    print("")

# Uji normalitas untuk masing-masing saham
for ticker in tickers:
    print(f"Uji Normalitas untuk {ticker}")
    jarque_bera_test(log_returns[ticker])


Uji Normalitas untuk AAPL
JB Test Statistic: 122.23955803743026, p-value: 2.8577039625908915e-27
Data tidak berdistribusi normal

Uji Normalitas untuk AMZN
JB Test Statistic: 174.95357380796247, p-value: 1.0216782617014645e-38
Data tidak berdistribusi normal

Uji Normalitas untuk META
JB Test Statistic: 4027.2447716734405, p-value: 0.0
Data tidak berdistribusi normal

Uji Normalitas untuk TSLA
JB Test Statistic: 205.93577553321083, p-value: 1.912557627186149e-45
Data tidak berdistribusi normal

Uji Normalitas untuk TSM
JB Test Statistic: 31.733395037418614, p-value: 1.285821909606062e-07
Data tidak berdistribusi normal

Uji Normalitas untuk IBM
JB Test Statistic: 1453.028552360038, p-value: 0.0
Data tidak berdistribusi normal

Uji Normalitas untuk BABA
JB Test Statistic: 45.599293875261644, p-value: 1.253831411157659e-10
Data tidak berdistribusi normal



# Cornish-Fisher Expansion

In [10]:
# Langkah 1: Hitung log returns
# log_returns = np.log(stocks_df['AAPL'] / stocks_df['AAPL'].shift(1)).dropna()

# Langkah 2: Hitung skewness dan kurtosis untuk setiap saham
skewness = log_returns.apply(skew)
kurt = log_returns.apply(kurtosis)

# Langkah 3: Tentukan level kepercayaan (misalnya, 99%)
alpha = 0.05
t = 1

z_normal = norm.ppf(1 - alpha)  # Quantile untuk distribusi normal

# Langkah 4: Cornish-Fisher Expansion untuk perbaikan kuantil
def cornish_fisher(z_normal, skewness, kurt):
    """
    Menggunakan Cornish-Fisher Expansion untuk memperbaiki quantile distribusi normal.
    """
    z_cornish_fisher = z_normal + (1 / 6) * (z_normal**2 - 1) * skewness + \
                       (1 / 24) * (z_normal**3 - 3 * z_normal) * kurt - \
                       (1 / 36) * (2 * z_normal**3 - 5 * z_normal) * skewness**2
    return z_cornish_fisher

# Langkah 5: Hitung VaR untuk setiap saham
var_normal = {}
var_cornish_fisher = {}

for ticker in tickers:
    # Ambil log returns untuk ticker tersebut
    returns = log_returns[ticker]
    
    # Hitung standar deviasi log returns
    std_dev = returns.std()
    
    # Hitung VaR dengan distribusi normal
    var_normal[ticker] = -z_normal * std_dev * np.sqrt(t)
    
    # Hitung VaR dengan Cornish-Fisher Expansion
    z_cornish_fisher_val = cornish_fisher(z_normal, skewness[ticker], kurt[ticker])
    var_cornish_fisher[ticker] = -z_cornish_fisher_val * std_dev * np.sqrt(t)

# Tampilkan hasil VaR
var_df = pd.DataFrame({
    'VaR_Normal': var_normal,
    'VaR_Cornish_Fisher': var_cornish_fisher
})

var_df

,VaR_Normal,VaR_Cornish_Fisher
AAPL,-0.023362,-0.023949
AMZN,-0.028577,-0.026111
META,-0.036700,-0.036752
TSLA,-0.062712,-0.066404
TSM,-0.040698,-0.041624
IBM,-0.023577,-0.019534
BABA,-0.038161,-0.039649


In [11]:
# Modal Awal
V0 = 10000  # Modal awal yang diinvestasikan

In [12]:
# Langkah 7: Hitung VaR dalam nilai uang
var_df['VaR Normal (USD)'] = V0 * (var_df['VaR_Normal'])
var_df['VaR Cornish-Fisher (USD)'] = V0 * (var_df['VaR_Cornish_Fisher'])
var_df

,VaR_Normal,VaR_Cornish_Fisher,VaR Normal (USD),VaR Cornish-Fisher (USD)
AAPL,-0.023362,-0.023949,-233.621174,-239.493383
AMZN,-0.028577,-0.026111,-285.771807,-261.107992
META,-0.036700,-0.036752,-366.997038,-367.520849
TSLA,-0.062712,-0.066404,-627.121543,-664.037536
TSM,-0.040698,-0.041624,-406.980608,-416.243912
IBM,-0.023577,-0.019534,-235.774346,-195.343164
BABA,-0.038161,-0.039649,-381.605823,-396.489577


# ARCH

In [13]:
# Langkah 4: Model ARCH/GARCH
# Memodelkan ARCH (GARCH) untuk masing-masing saham
def arch_garch_model(data):
    model = arch_model(data, vol='ARCH', p=1, rescale=False)
    model_fit = model.fit(disp="off")
    print(model_fit.summary())
    print("")
    return model_fit

# Modelkan ARCH/GARCH untuk setiap saham
for ticker in tickers:
    print("==============================================================================")
    print(f"=========================== Model ARCH untuk {ticker} ============================")
    print("==============================================================================")
    result = arch_garch_model(log_returns[ticker])

    # Output hasil estimasi model ARCH
    print(result.summary())

    # Langkah 4: Mengambil Residuals dari Model
    residuals = result.resid

    # Langkah 5: Uji Heteroskedastisitas dengan ARCH LM Test
    # Uji heteroskedastisitas (ARCH LM Test) pada residuals
    arch_test = het_arch(residuals)
    print(f"\nARCH LM Test p-value: {arch_test[1]}\n")

    # Jika p-value < 0.05, maka ada bukti adanya heteroskedastisitas pada residuals
    if arch_test[1] < 0.05:
        print("Terdapat heteroskedastisitas pada residuals.")
    else:
        print("Tidak terdapat heteroskedastisitas pada residuals.")
    print("\n")



=========================== Model ARCH untuk AAPL ============================
                      Constant Mean - ARCH Model Results                      
Dep. Variable:                   AAPL   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:                712.787
Distribution:                  Normal   AIC:                          -1419.57
Method:            Maximum Likelihood   BIC:                          -1409.01
                                        No. Observations:                  250
Date:                Fri, Dec 06 2024   Df Residuals:                      249
Time:                        17:24:52   Df Model:                            1
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
------------------------------------------------------